In [69]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [70]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [71]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [72]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [73]:
from bs4 import BeautifulSoup
url = requests.get('https://www.districtcouncils.gov.hk/index.html').text
soup = BeautifulSoup(url, 'html.parser')


In [74]:
table=[]
cell={}
for i in soup.find_all('div', class_="hk_btn index_hk_district"):
    if i.find("div", {"class": "tag_hk"}) != None: 
        cell['Area']=i.find("div", {"class": "tag_hk"}).text.strip()
    for a in i.find_all('a'):
        cell['Districts']=a.text.strip()
        cell_copy=cell.copy()
        table.append(cell_copy)
for i in soup.find_all('div', class_="kin_btn index_kl_district"):
    if i.find("div", {"class": "tag_kln"}) != None: 
        cell['Area']=i.find("div", {"class": "tag_kln"}).text.strip()
    for a in i.find_all('a'):
        cell['Districts']=a.text.strip()
        cell_copy=cell.copy()
        table.append(cell_copy)
for i in soup.find_all('div', class_="nt_btn index_nt_district"):
    if i.find("div", {"class": "tag_nt"}) != None: 
        cell['Area']=i.find("div", {"class": "tag_nt"}).text.strip()
    for a in i.find_all('a'):
        cell['Districts']=a.text.strip()
        cell_copy=cell.copy()
        table.append(cell_copy)

In [75]:
table

[{'Area': 'Hong Kong', 'Districts': 'Central & Western'},
 {'Area': 'Hong Kong', 'Districts': 'Eastern'},
 {'Area': 'Hong Kong', 'Districts': 'Southern'},
 {'Area': 'Hong Kong', 'Districts': 'Wan Chai'},
 {'Area': 'Kowloon', 'Districts': 'Kowloon City'},
 {'Area': 'Kowloon', 'Districts': 'Kwun Tong'},
 {'Area': 'Kowloon', 'Districts': 'Sham Shui Po'},
 {'Area': 'Kowloon', 'Districts': 'Yau Tsim Mong'},
 {'Area': 'Kowloon', 'Districts': 'Wong Tai Sin'},
 {'Area': 'New Territories', 'Districts': 'Islands'},
 {'Area': 'New Territories', 'Districts': 'Kwai Tsing'},
 {'Area': 'New Territories', 'Districts': 'North'},
 {'Area': 'New Territories', 'Districts': 'Sai Kung'},
 {'Area': 'New Territories', 'Districts': 'Sha Tin'},
 {'Area': 'New Territories', 'Districts': 'Tai Po'},
 {'Area': 'New Territories', 'Districts': 'Tsuen Wan'},
 {'Area': 'New Territories', 'Districts': 'Tuen Mun'},
 {'Area': 'New Territories', 'Districts': 'Yuen Long'}]

In [76]:
df = pd.DataFrame.from_dict(table)

#Islands, Southern and North return incorrect latitude & longitude
df['Districts'].replace('Islands','Islands District',inplace=True)
df['Districts'].replace('Southern','Southern District',inplace=True)
df['Districts'].replace('North','North District',inplace=True)

In [77]:
df

,Area,Districts
0,Hong Kong,Central & Western
1,Hong Kong,Eastern
2,Hong Kong,Southern District
3,Hong Kong,Wan Chai
4,Kowloon,Kowloon City
5,Kowloon,Kwun Tong
6,Kowloon,Sham Shui Po
7,Kowloon,Yau Tsim Mong
8,Kowloon,Wong Tai Sin
9,New Territories,Islands District


In [78]:
address = 'Islands District, Hong Kong'

geolocator = Nominatim(user_agent="hk_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 22.23007565, 113.98678546827479.


In [79]:
Latitude = []
Longitude = []
for i in range(len(df)):
    address = df['Districts'][i]+', Hong Kong'

    geolocator = Nominatim(user_agent="hk_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude

    Latitude.append(latitude)
    Longitude.append(longitude)
       
df["Latitude"] = Latitude
df["Longitude"] = Longitude

In [80]:
df

,Area,Districts,Latitude,Longitude
0,Hong Kong,Central & Western,22.281829,114.158278
1,Hong Kong,Eastern,22.280348,114.223472
2,Hong Kong,Southern District,22.219269,114.225223
3,Hong Kong,Wan Chai,22.279015,114.172483
4,Kowloon,Kowloon City,22.330160,114.189937
5,Kowloon,Kwun Tong,22.312937,114.225610
6,Kowloon,Sham Shui Po,22.328190,114.160854
7,Kowloon,Yau Tsim Mong,22.303165,114.160212
8,Kowloon,Wong Tai Sin,22.341654,114.193859
9,New Territories,Islands District,22.230076,113.986785


In [81]:
#No venue found in Southern District, use Latitude & Longitude(22.247222, 114.158889) of Southern District from Google
df.at[2,'Latitude']=22.247222
df.at[2,'Longitude']=114.158889

In [82]:
df

,Area,Districts,Latitude,Longitude
0,Hong Kong,Central & Western,22.281829,114.158278
1,Hong Kong,Eastern,22.280348,114.223472
2,Hong Kong,Southern District,22.247222,114.158889
3,Hong Kong,Wan Chai,22.279015,114.172483
4,Kowloon,Kowloon City,22.330160,114.189937
5,Kowloon,Kwun Tong,22.312937,114.225610
6,Kowloon,Sham Shui Po,22.328190,114.160854
7,Kowloon,Yau Tsim Mong,22.303165,114.160212
8,Kowloon,Wong Tai Sin,22.341654,114.193859
9,New Territories,Islands District,22.230076,113.986785


In [83]:
# create map of all districts using latitude and longitude values
map_districts = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, area, districts in zip(df['Latitude'], df['Longitude'], df['Area'], df['Districts']):
    label = '{}, {}'.format(districts, area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_districts)  
    
map_districts

In [84]:
CLIENT_ID = 'HA2AK24L51QJH1TADKA503ASJ1BNCMY0SW54IJ1WIETVCSZW' # your Foursquare ID
CLIENT_SECRET = '125GVJJ1OC2ZRNSBPJQLZCJ0XS0K4OE4QRJX2PYS25YF0TV0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [85]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url
# display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=HA2AK24L51QJH1TADKA503ASJ1BNCMY0SW54IJ1WIETVCSZW&client_secret=125GVJJ1OC2ZRNSBPJQLZCJ0XS0K4OE4QRJX2PYS25YF0TV0&v=20180605&ll=22.4444901,114.0287645&radius=500&limit=100'

In [86]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60c606fd5928183be01ee654'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Yuen Long',
  'headerFullLocation': 'Yuen Long, Hong Kong',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 22.448990104500005,
    'lng': 114.03362422754664},
   'sw': {'lat': 22.439990095499994, 'lng': 114.02390477245335}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5528eb27498e090969b5b78d',
       'name': 'Sushi Man (鮨文)',
       'location': {'address': 'G/F, 5 Yan Lok Square',
        'lat': 22.44348053472518,
        'lng': 114.02671390918015,
        'labeledLatLngs': [{'label': 'display',
          'la

In [87]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [88]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sushi Man (鮨文),Japanese Restaurant,22.443481,114.026714
1,亞玉豆腐花,Dessert Shop,22.444035,114.031049
2,Aya (彩),Ramen Restaurant,22.441018,114.030366
3,過橋麵檔,Chinese Restaurant,22.441612,114.030452
4,Hang Heung (恆香老餅家),Bakery,22.444397,114.030241


In [89]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [90]:
districts_venues = getNearbyVenues(names=df['Districts'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Central & Western
Eastern
Southern District
Wan Chai
Kowloon City
Kwun Tong
Sham Shui Po
Yau Tsim Mong
Wong Tai Sin
Islands District
Kwai Tsing
North District
Sai Kung
Sha Tin
Tai Po
Tsuen Wan
Tuen Mun
Yuen Long


In [91]:
districts_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central & Western,22.281829,114.158278,Mandarin Grill + Bar (文華扒房＋酒吧),22.281928,114.159408,Steakhouse
1,Central & Western,22.281829,114.158278,Mandarin Oriental Hong Kong (香港文華東方酒店),22.281857,114.159382,Hotel
2,Central & Western,22.281829,114.158278,The Mandarin Cake Shop,22.281959,114.159416,Bakery
3,Central & Western,22.281829,114.158278,Mott 32 (卅二公館),22.280286,114.159080,Dim Sum Restaurant
4,Central & Western,22.281829,114.158278,Sift Patisserie,22.280922,114.159885,Cupcake Shop


In [92]:
districts_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central & Western,92,92,92,92,92,92
Eastern,18,18,18,18,18,18
Islands District,2,2,2,2,2,2
Kowloon City,56,56,56,56,56,56
Kwai Tsing,3,3,3,3,3,3
Kwun Tong,55,55,55,55,55,55
North District,2,2,2,2,2,2
Sai Kung,48,48,48,48,48,48
Sha Tin,47,47,47,47,47,47


In [93]:
# one hot encoding
districts_onehot = pd.get_dummies(districts_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
districts_onehot['Neighborhood'] = districts_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [districts_onehot.columns[-1]] + list(districts_onehot.columns[:-1])
districts_onehot = districts_onehot[fixed_columns]

districts_onehot.head()

,Neighborhood,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beer Store,Beijing Restaurant,Bookstore,Boutique,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cultural Center,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Dumpling Restaurant,Electronics Store,English Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,German Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Hong Kong Restaurant,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean BBQ Restaurant,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Men's Store,Metro Station,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Mountain,Multiplex,Noodle House,Pakistani Restaurant,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,River,Rock Climbing Spot,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shopping Mall,Skating Rink,Snack Place,Social Club,Spa,Spanish Restaurant,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theme Park,Toy / Game Store,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfall,Wine Shop,Yoga Studio
0,Central & Western,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Central & Western,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Central & Western,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Central & Western,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Central & Western,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [94]:
districts_grouped = districts_onehot.groupby('Neighborhood').mean().reset_index()
districts_grouped

,Neighborhood,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beer Store,Beijing Restaurant,Bookstore,Boutique,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cultural Center,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Dumpling Restaurant,Electronics Store,English Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,German Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Hong Kong Restaurant,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean BBQ Restaurant,Korean Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Men's Store,Metro Station,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Mountain,Multiplex,Noodle House,Pakistani Restaurant,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,River,Rock Climbing Spot,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shopping Mall,Skating Rink,Snack Place,Social Club,Spa,Spanish Restaurant,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theme Park,Toy / Game Store,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfall,Wine Shop,Yoga Studio
0,Central & Western,0.000000,0.010870,0.021739,0.01087,0.000000,0.000000,0.010870,0.021739,0.054348,0.01087,0.000000,0.01087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.032609,0.010870,0.000000,0.021739,0.000000,0.010870,0.054348,0.000000,0.000000,0.000000,0.01087,0.01087,0.010870,0.000000,0.010870,0.021739,0.010870,0.010870,0.01087,0.000000,0.000000,0.000000,0.000000,0.000000,0.032609,0.010870,0.000000,0.010870,0.000000,0.032609,0.032609,0.000000,0.000000,0.000000,0.032609,0.021739,0.000000,0.000000,0.000000,0.021739,0.000000,0.032609,0.076087,0.01087,0.000000,0.000000,0.000000,0.010870,0.000000,0.01087,0.021739,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.0,0.010870,0.000000,0.01087,0.000000,0.000000,0.000000,0.01087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01087,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.01087,0.010870,0.000000,0.000000,0.010870,0.010870,0.000000,0.010870,0.021739,0.032609,0.000000,0.000000,0.000000,0.010870,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.0,0.000000,0.021739
1,Eastern,0.000000,0.000000,0.000000,0.00000,0.055556,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.055556,0.055556,0.055556,0.00000,0.00000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.00000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,

In [95]:
num_top_venues = 5

for hood in districts_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = districts_grouped[districts_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central & Western----
                 venue  freq
0  Japanese Restaurant  0.08
1         Cocktail Bar  0.05
2                  Bar  0.05
3    French Restaurant  0.03
4                  Spa  0.03


----Eastern----
                  venue  freq
0  Fast Food Restaurant  0.17
1         Shopping Mall  0.17
2  Cantonese Restaurant  0.06
3    Chinese Restaurant  0.06
4           Pizza Place  0.06


----Islands District----
                venue  freq
0  Rock Climbing Spot   0.5
1            Mountain   0.5
2               Plaza   0.0
3        Perfume Shop   0.0
4            Pharmacy   0.0


----Kowloon City----
                   venue  freq
0        Thai Restaurant  0.21
1           Dessert Shop  0.16
2                   Café  0.07
3  Vietnamese Restaurant  0.05
4           Noodle House  0.04


----Kwai Tsing----
                 venue  freq
0                 Café  0.33
1   Chinese Restaurant  0.33
2               Tunnel  0.33
3  American Restaurant  0.00
4                Plaza  0.00


-

In [96]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = districts_grouped['Neighborhood']

for ind in np.arange(districts_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(districts_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central & Western,Japanese Restaurant,Cocktail Bar,Bar,French Restaurant,Spa,Gym,Gym / Fitness Center,Italian Restaurant,Hotel,Cantonese Restaurant
1,Eastern,Fast Food Restaurant,Shopping Mall,Cantonese Restaurant,Chinese Restaurant,Pizza Place,Coffee Shop,Convenience Store,Cultural Center,Indian Restaurant,Toy / Game Store
2,Islands District,Rock Climbing Spot,Mountain,Plaza,Perfume Shop,Pharmacy,Pier,Pizza Place,Playground,Pool,Pastry Shop
3,Kowloon City,Thai Restaurant,Dessert Shop,Café,Vietnamese Restaurant,Noodle House,Halal Restaurant,Coffee Shop,Chinese Restaurant,Bakery,Cha Chaan Teng
4,Kwai Tsing,Café,Chinese Restaurant,Tunnel,American Restaurant,Plaza,Pharmacy,Pier,Pizza Place,Playground,Pool


In [98]:
# set number of clusters
kclusters = 5

districts_grouped_clustering = districts_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(districts_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/srv/conda/envs/notebook/lib/python3.7/site-

array([1, 0, 3, 1, 4, 1, 2, 1, 1, 0], dtype=int32)

In [99]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [100]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Central & Western,Japanese Restaurant,Cocktail Bar,Bar,French Restaurant,Spa,Gym,Gym / Fitness Center,Italian Restaurant,Hotel,Cantonese Restaurant
1,0,Eastern,Fast Food Restaurant,Shopping Mall,Cantonese Restaurant,Chinese Restaurant,Pizza Place,Coffee Shop,Convenience Store,Cultural Center,Indian Restaurant,Toy / Game Store
2,3,Islands District,Rock Climbing Spot,Mountain,Plaza,Perfume Shop,Pharmacy,Pier,Pizza Place,Playground,Pool,Pastry Shop
3,1,Kowloon City,Thai Restaurant,Dessert Shop,Café,Vietnamese Restaurant,Noodle House,Halal Restaurant,Coffee Shop,Chinese Restaurant,Bakery,Cha Chaan Teng
4,4,Kwai Tsing,Café,Chinese Restaurant,Tunnel,American Restaurant,Plaza,Pharmacy,Pier,Pizza Place,Playground,Pool
5,1,Kwun Tong,Fast Food Restaurant,Cha Chaan Teng,Coffee Shop,Chinese Restaurant,Restaurant,Café,Clothing Store,Noodle House,Cantonese Restaurant,Sushi Restaurant
6,2,North District,River,Waterfall,American Restaurant,Plaza,Perfume Shop,Pharmacy,Pier,Pizza Place,Playground,Portuguese Restaurant
7,1,Sai Kung,Café,Seafood Restaurant,Dessert Shop,Burger Joint,Thai Restaurant,Pizza Place,Food Court,Sri Lankan Restaurant,Taco Place,Indian Restaurant
8,1,Sha Tin,Shopping Mall,Chinese Restaurant,Café,Japanese Restaurant,Coffee Shop,Asian Restaurant,Fast Food Restaurant,Pool,Ramen Restaurant,Clothing Store
9,0,Sham Shui Po,Noodle House,Shopping Mall,Snack Place,Fast Food Restaurant,Dessert Shop,Cha Chaan Teng,Indonesian Restaurant,Flea Market,Playground,Hong Kong Restaurant


In [101]:
districts_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
districts_merged = districts_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Districts')

districts_merged.head() # check the last columns!

,Area,Districts,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hong Kong,Central & Western,22.281829,114.158278,1,Japanese Restaurant,Cocktail Bar,Bar,French Restaurant,Spa,Gym,Gym / Fitness Center,Italian Restaurant,Hotel,Cantonese Restaurant
1,Hong Kong,Eastern,22.280348,114.223472,0,Fast Food Restaurant,Shopping Mall,Cantonese Restaurant,Chinese Restaurant,Pizza Place,Coffee Shop,Convenience Store,Cultural Center,Indian Restaurant,Toy / Game Store
2,Hong Kong,Southern District,22.247222,114.158889,1,Cha Chaan Teng,Market,Park,Food Court,Dessert Shop,Coffee Shop,Hotel,Chinese Restaurant,Sushi Restaurant,Taiwanese Restaurant
3,Hong Kong,Wan Chai,22.279015,114.172483,1,Italian Restaurant,Café,Japanese Restaurant,Coffee Shop,Spanish Restaurant,Hotel,Bakery,Hotpot Restaurant,Chinese Restaurant,Cantonese Restaurant
4,Kowloon,Kowloon City,22.330160,114.189937,1,Thai Restaurant,Dessert Shop,Café,Vietnamese Restaurant,Noodle House,Halal Restaurant,Coffee Shop,Chinese Restaurant,Bakery,Cha Chaan Teng


In [102]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(districts_merged['Latitude'], districts_merged['Longitude'], districts_merged['Districts'], districts_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [103]:
districts_merged.loc[districts_merged['Cluster Labels'] == 0, districts_merged.columns[[1] + list(range(5, districts_merged.shape[1]))]]

,Districts,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Eastern,Fast Food Restaurant,Shopping Mall,Cantonese Restaurant,Chinese Restaurant,Pizza Place,Coffee Shop,Convenience Store,Cultural Center,Indian Restaurant,Toy / Game Store
6,Sham Shui Po,Noodle House,Shopping Mall,Snack Place,Fast Food Restaurant,Dessert Shop,Cha Chaan Teng,Indonesian Restaurant,Flea Market,Playground,Hong Kong Restaurant
14,Tai Po,Fast Food Restaurant,Hong Kong Restaurant,Noodle House,Chinese Restaurant,Department Store,Pub,Coffee Shop,Plaza,Sushi Restaurant,Thai Restaurant
17,Yuen Long,Chinese Restaurant,Noodle House,Fast Food Restaurant,Dessert Shop,Shopping Mall,Hong Kong Restaurant,Japanese Restaurant,Thai Restaurant,Cantonese Restaurant,Dumpling Restaurant


In [104]:
districts_merged.loc[districts_merged['Cluster Labels'] == 1, districts_merged.columns[[1] + list(range(5, districts_merged.shape[1]))]]

,Districts,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central & Western,Japanese Restaurant,Cocktail Bar,Bar,French Restaurant,Spa,Gym,Gym / Fitness Center,Italian Restaurant,Hotel,Cantonese Restaurant
2,Southern District,Cha Chaan Teng,Market,Park,Food Court,Dessert Shop,Coffee Shop,Hotel,Chinese Restaurant,Sushi Restaurant,Taiwanese Restaurant
3,Wan Chai,Italian Restaurant,Café,Japanese Restaurant,Coffee Shop,Spanish Restaurant,Hotel,Bakery,Hotpot Restaurant,Chinese Restaurant,Cantonese Restaurant
4,Kowloon City,Thai Restaurant,Dessert Shop,Café,Vietnamese Restaurant,Noodle House,Halal Restaurant,Coffee Shop,Chinese Restaurant,Bakery,Cha Chaan Teng
5,Kwun Tong,Fast Food Restaurant,Cha Chaan Teng,Coffee Shop,Chinese Restaurant,Restaurant,Café,Clothing Store,Noodle House,Cantonese Restaurant,Sushi Restaurant
7,Yau Tsim Mong,Chinese Restaurant,Japanese Restaurant,Café,Italian Restaurant,Hotel,Cantonese Restaurant,Spa,Skating Rink,Sushi Restaurant,Clothing Store
8,Wong Tai Sin,Fast Food Restaurant,Coffee Shop,Szechuan Restaurant,Pool,Hong Kong Restaurant,Park,Burger Joint,Tea Room,Temple,Cha Chaan Teng
12,Sai Kung,Café,Seafood Restaurant,Dessert Shop,Burger Joint,Thai Restaurant,Pizza Place,Food Court,Sri Lankan Restaurant,Taco Place,Indian Restaurant
13,Sha Tin,Shopping Mall,Chinese Restaurant,Café,Japanese Restaurant,Coffee Shop,Asian Restaurant,Fast Food Restaurant,Pool,Ramen Restaurant,Clothing Store
15,Tsuen Wan,Cha Chaan Teng,Shopping Mall,Hong Kong Restaurant,Coffee Shop,Japanese Restaurant,Chinese Restaurant,Fast Food Restaurant,Shanghai Restaurant,Department Store,Café


In [105]:
districts_merged.loc[districts_merged['Cluster Labels'] == 2, districts_merged.columns[[1] + list(range(5, districts_merged.shape[1]))]]

,Districts,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,North District,River,Waterfall,American Restaurant,Plaza,Perfume Shop,Pharmacy,Pier,Pizza Place,Playground,Portuguese Restaurant


In [106]:
districts_merged.loc[districts_merged['Cluster Labels'] == 3, districts_merged.columns[[1] + list(range(5, districts_merged.shape[1]))]]

,Districts,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Islands District,Rock Climbing Spot,Mountain,Plaza,Perfume Shop,Pharmacy,Pier,Pizza Place,Playground,Pool,Pastry Shop


In [107]:
districts_merged.loc[districts_merged['Cluster Labels'] == 4, districts_merged.columns[[1] + list(range(5, districts_merged.shape[1]))]]

,Districts,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Kwai Tsing,Café,Chinese Restaurant,Tunnel,American Restaurant,Plaza,Pharmacy,Pier,Pizza Place,Playground,Pool
